In [1]:
%pylab inline
import time
import numpy as np
from segbench import csaAssign, correspondPixels, matchEdgeMaps2D, matchEdgeMaps3D

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = np.array([[1, 4, 3],
    [2, 5, 3],
    [3, 6, 3],
    [1, 5, 1],
    [2, 6, 1],
    [3, 4, 5]])
n = 6
e1 = csaAssign(n,data)
print e1

[[ 1.  5.  1.]
 [ 2.  6.  1.]
 [ 3.  4.  5.]]


In [3]:
n = 1000
dg = np.random.rand(n,n)>.5
m = np.sum(dg.flat)
i = np.array(dg.ravel().nonzero()).astype(float)
g1 = 1+np.floor(np.divide(i,n))
g2 = (1+np.floor(np.divide(i,n)),
     1+ np.mod(i,n)+n,
     1+np.floor(np.random.rand(1,m))*1000)
g2 = np.array(np.vstack(g2)).T
t = time.time()
e2 = csaAssign(n*2,g2)
print '{:.3e}s'.format(time.time() - t)
esum = np.sum(e2,axis=0)
if not all(esum[0:2]==(n*(n+1)/2,n*(n+1)/2+n*n)): raise ValueError("bug")
if not np.sum(esum[0:2])==(2*n*(2*n+1)/2): raise ValueError("bug")
print n,m, e2[3][:]

1.022e-01s
1000 500149 [  4.00000000e+00   1.99600000e+03   1.00000000e+00]


In [4]:
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,10):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)

        match1 = zeros_like(bmap1)
        match2 = zeros_like(bmap2)
        t = time.time()
        cost,oc,match1,match2 = correspondPixels(bmap1,bmap2,.005)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print npix,ncor,nout
        if ncor+nout!=npix: raise ValueError("bug")
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T


        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 4.81e-03s 2 2 0
n=1 thresh=1.11e-01 4.60e-05s 2 2 0
n=1 thresh=2.22e-01 4.10e-05s 2 2 0
n=1 thresh=3.33e-01 2.10e-05s 1 0 1
n=1 thresh=4.44e-01 2.10e-05s 0 0 0
n=1 thresh=5.56e-01 2.22e-05s 0 0 0
n=1 thresh=6.67e-01 2.10e-05s 1 0 1
n=1 thresh=7.78e-01 2.00e-05s 0 0 0
n=1 thresh=8.89e-01 2.00e-05s 1 0 1
n=1 thresh=1.00e+00 1.98e-05s 0 0 0
n=2 thresh=0.00e+00 7.89e-05s 8 8 0
n=2 thresh=1.11e-01 5.60e-05s 7 6 1
n=2 thresh=2.22e-01 7.10e-05s 8 8 0
n=2 thresh=3.33e-01 5.48e-05s 7 6 1
n=2 thresh=4.44e-01 4.82e-05s 6 4 2
n=2 thresh=5.56e-01 2.10e-05s 1 0 1
n=2 thresh=6.67e-01 2.00e-05s 2 0 2
n=2 thresh=7.78e-01 2.10e-05s 0 0 0
n=2 thresh=8.89e-01 2.10e-05s 2 0 2
n=2 thresh=1.00e+00 2.38e-05s 0 0 0
n=3 thresh=0.00e+00 1.71e-04s 18 18 0
n=3 thresh=1.11e-01 1.48e-04s 17 16 1
n=3 thresh=2.22e-01 1.11e-04s 14 12 2
n=3 thresh=3.33e-01 4.82e-05s 10 4 6
n=3 thresh=4.44e-01 2.38e-05s 6 0 6
n=3 thresh=5.56e-01 4.20e-05s 4 2 2
n=3 thresh=6.67e-01 2.19e-05s 3 0 3
n=3 thresh=7.78e-01 5

In [5]:
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,2):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)
        t = time.time()
        cost,match1,match2 = matchEdgeMaps2D(bmap1,bmap2,.005,100)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T


        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 7.70e-05s 2 + 0 = 2 == 2
n=1 thresh=1.00e+00 8.11e-05s 0 + 0 = 0 == 0
n=2 thresh=0.00e+00 1.89e-04s 8 + 0 = 8 == 8
n=2 thresh=1.00e+00 7.39e-05s 0 + 0 = 0 == 0
n=3 thresh=0.00e+00 3.99e-04s 18 + 0 = 18 == 18
n=3 thresh=1.00e+00 6.39e-05s 0 + 0 = 0 == 0
n=5 thresh=0.00e+00 5.45e-04s 50 + 0 = 50 == 50
n=5 thresh=1.00e+00 8.20e-05s 0 + 0 = 0 == 0
n=8 thresh=0.00e+00 1.79e-03s 128 + 0 = 128 == 128
n=8 thresh=1.00e+00 4.41e-05s 0 + 0 = 0 == 0
n=13 thresh=0.00e+00 5.38e-03s 338 + 0 = 338 == 338
n=13 thresh=1.00e+00 5.01e-05s 0 + 0 = 0 == 0
n=22 thresh=0.00e+00 1.69e-02s 966 + 2 = 968 == 968
n=22 thresh=1.00e+00 7.01e-05s 0 + 0 = 0 == 0
n=36 thresh=0.00e+00 5.22e-02s 2582 + 10 = 2592 == 2592
n=36 thresh=1.00e+00 1.49e-04s 0 + 0 = 0 == 0
n=60 thresh=0.00e+00 1.75e-01s 7182 + 18 = 7200 == 7200
n=60 thresh=1.00e+00 3.40e-04s 0 + 0 = 0 == 0
n=100 thresh=0.00e+00 6.34e-01s 19952 + 48 = 20000 == 20000
n=100 thresh=1.00e+00 8.62e-04s 0 + 0 = 0 == 0


In [6]:
for n in np.round(np.logspace(0,6,4,base=2)).astype(int):
    for thresh in np.linspace(.5,.9,4):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n,n)>thresh).astype(float)

        t = time.time()
        cost,match1,match2 = matchEdgeMaps3D(bmap1,bmap2,.000005,100)
        print '{:1.4f}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T


        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if sum(a-b): raise ValueError("bug")

n=1 thresh=5.00e-01 0.0001s 0 + 1 = 1 == 1
n=1 thresh=6.33e-01 0.0000s 0 + 1 = 1 == 1
n=1 thresh=7.67e-01 0.0001s 0 + 1 = 1 == 1
n=1 thresh=9.00e-01 0.0000s 0 + 0 = 0 == 0
n=4 thresh=5.00e-01 0.0003s 22 + 32 = 54 == 54
n=4 thresh=6.33e-01 0.0003s 22 + 29 = 51 == 51
n=4 thresh=7.67e-01 0.0001s 4 + 25 = 29 == 29
n=4 thresh=9.00e-01 0.0000s 0 + 11 = 11 == 11
n=16 thresh=5.00e-01 0.0492s 2080 + 2005 = 4085 == 4085
n=16 thresh=6.33e-01 0.0221s 1088 + 1909 = 2997 == 2997
n=16 thresh=7.67e-01 0.0083s 474 + 1417 = 1891 == 1891
n=16 thresh=9.00e-01 0.0016s 98 + 704 = 802 == 802
n=64 thresh=5.00e-01 10.0760s 130904 + 131269 = 262173 == 262173
n=64 thresh=6.33e-01 3.4448s 69638 + 122447 = 192085 == 192085
n=64 thresh=7.67e-01 1.1007s 28796 + 93989 = 122785 == 122785
n=64 thresh=9.00e-01 0.1713s 5264 + 47035 = 52299 == 52299
